In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../DATASETS_TRACTATS/TOT_fzv_1415_altitud.csv")

In [3]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [5]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "ALTITUD", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df = encoder.fit_transform(df)

In [23]:
df = df[df["CAMPAÑA"] != "22"].drop(columns ="SUPERFICIE")

In [24]:
X = df.drop(columns = ["PRODUCCION", "CAMPAÑA"])
y = df["PRODUCCION"]

In [26]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [100],
                            "learning_rate": [0.15],
                            "max_depth": [40],
                            "min_child_weight": [1],
                            "gamma":[0.1],
                            "colsample_bytree":[0.4],
                            "booster": ["dart"]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "neg_root_mean_squared_error", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

#### Random

In [27]:
model_random = grid(X,y)

In [28]:
model_random.best_score_

-6561.264679128365

In [29]:
pd.DataFrame(model_random.cv_results_).iloc[:, 14:25]

,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error
0,0.851431,0.765069,0.742548,0.7645,0.04526,1,-7580.233252,-6698.868021,-5153.99235,-6439.069352,-6934.160421


#### Testejant un any

In [30]:
years_to_test = ["14", "15", "16", "17", "18", "19", "20", "21"]
model_one_year_test = {}
for year in years_to_test:
    X_train = X[df["CAMPAÑA"] != year]
    y_train = y[df["CAMPAÑA"] != year]
    X_test = X[df["CAMPAÑA"] == year]
    y_test = y[df["CAMPAÑA"] == year]
    m_y = grid(X_train, y_train)
    score_y = m_y.score(X_test, y_test)
    model_one_year_test[year] = (m_y, score_y)

In [31]:
for year in years_to_test:
    print(year)
    print(pd.DataFrame(model_one_year_test[year][0].cv_results_).loc[:,["mean_test_r2", "mean_test_neg_root_mean_squared_error"]])
    print(model_one_year_test[year][1])

14
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.764717                           -6488.668975
-7724.804012517787
15
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.741791                           -6583.195512
-6957.818320739117
16
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.760578                           -6592.361971
-6701.060973035428
17
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.753102                           -6848.726009
-5575.655956133801
18
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.748361                            -6707.37225
-6038.470703206219
19
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.741626                           -7017.367042
-4785.4153814097535
20
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.765223                           -6572.281788
-6932.751360525485
21
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.756239  

#### Per anys

In [32]:
years_to_test2 = df["CAMPAÑA"].unique()
model_one_year = {}
for year in years_to_test2:
    X2 = X[df["CAMPAÑA"] == year]
    y2 = y[df["CAMPAÑA"] == year]
    m_y = grid(X2, y2)
    model_one_year[year] = m_y

In [33]:
for year in years_to_test2:
    print(year)
    print(pd.DataFrame(model_one_year[year].cv_results_).loc[:,["mean_test_r2", "mean_test_neg_root_mean_squared_error"]])

14
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.322994                          -13787.419996
15
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.036947                          -15389.601996
16
   mean_test_r2  mean_test_neg_root_mean_squared_error
0       -0.1666                          -13749.832358
17
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.049774                           -10938.53245
18
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.111593                          -14175.968036
19
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.054788                          -11258.730029
20
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.229936                           -13437.04554
21
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.365809                           -13812.90385


In [34]:
X

,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,ALTITUD_MIN,ALTITUD_DIF
0,1,1,1,1,1,1,1,1,650,10
1,2,1,1,1,2,2,1,1,650,10
2,2,1,1,1,3,2,1,1,650,10
3,3,1,1,1,2,2,1,1,650,10
4,3,1,1,1,4,2,1,2,650,10
...,...,...,...,...,...,...,...,...,...,...
7840,578,86,15,3,6,2,1,1,700,0
7841,579,86,15,3,2,2,1,1,700,0
7842,579,86,15,3,1,2,1,1,700,0
7843,580,86,15,3,3,2,1,1,700,0
